In [28]:
print "Importing..."
import os
import io
from pandas import DataFrame
import numpy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.cross_validation import KFold
from sklearn.metrics import confusion_matrix, f1_score
import sys
from sklearn.ensemble import RandomForestClassifier
print "Import done..."

Importing...
Import done...


In [32]:
NEWLINE = '\n'
SKIP_FILES = {'cmds'}

def read_files(path):
    for root, dir_names, file_names in os.walk(path):
        for path in dir_names:
            read_files(os.path.join(root, path))
        for file_name in file_names:
            if file_name not in SKIP_FILES:
                file_path = os.path.join(root, file_name)
                if os.path.isfile(file_path):
                    past_header, lines = False, []
                    f = io.open(file_path, encoding="latin-1")
                    for line in f:
                        if past_header:
                            lines.append(line)
                        elif line == NEWLINE:
                            past_header = True
                    f.close()
                    content = NEWLINE.join(lines)
                    yield file_path, content

def build_data_frame(path, classification):
    rows = []
    index = []
    read_all = True
    mails_to_read = 500
    count = 1
    for file_name, text in read_files(path):
        if not read_all:
            if count <= mails_to_read:
                count += 1
                rows.append({'text': text, 'class': classification})
                index.append(file_name)
            else:
                break
        else:
#             print "Reading all..."
            rows.append({'text': text, 'class': classification})
            index.append(file_name)

    data_frame = DataFrame(rows, index=index)
    return data_frame

HAM = 'ham'
SPAM = 'spam'
# ('data/spam',        SPAM),
#     ('data/easy_ham',    HAM),
#     ('data/hard_ham',    HAM),
SOURCES = [
    ('D:\sasnjeev\enron_data\beck-s',      HAM),
    ('D:\sasnjeev\emron_data\farmer-d',    HAM),
    ('D:\sasnjeev\enron_data\kaminski-v',  HAM),
    ('D:\sasnjeev\enron_data\kitchen-l',   HAM),
    ('D:\sasnjeev\enron_data\lokay-m',     HAM),
    ('D:\sasnjeev\enron_data\williams-w3', HAM),
    ('D:\sasnjeev\enron_data\BG',          SPAM),
    ('D:\sasnjeev\enron_data\GP',          SPAM),
    ('D:\sasnjeev\enron_data\SH',          SPAM)
]
print "Reading mails..."
data = DataFrame({'text': [], 'class': []})
count = 1

for path, classification in SOURCES:
    print "Reading "+path
    data = data.append(build_data_frame(path, classification))
    

data = data.reindex(numpy.random.permutation(data.index))
print "Reindexing done. Dataframe ready..."

Reading mails...
Reading D:\sasnjeev\enron_dateck-s
Reading D:\sasnjeev\emron_dataarmer-d
Reading D:\sasnjeev\enron_data\kaminski-v
Reading D:\sasnjeev\enron_data\kitchen-l
Reading D:\sasnjeev\enron_data\lokay-m
Reading D:\sasnjeev\enron_data\williams-w3
Reading D:\sasnjeev\enron_data\BG
Reading D:\sasnjeev\enron_data\GP
Reading D:\sasnjeev\enron_data\SH
Reindexing done. Dataframe ready...


In [33]:
k_fold = KFold(n=len(data), n_folds=6)
scores = []
confusion = numpy.array([[0, 0], [0, 0]])
for train_indices, test_indices in k_fold:
#     print len(train_indices), len(test_indices)
    train_text = data.iloc[train_indices]['text'].values
    train_y = data.iloc[train_indices]['class'].values

    test_text = data.iloc[test_indices]['text'].values
    test_y = data.iloc[test_indices]['class'].values
    count_vectorizer = CountVectorizer()
    print "Fitting..."
    counts = count_vectorizer.fit_transform(train_text)
#     print counts
#     classifier = MultinomialNB()
#     targets = data['class'].values
    classifier = RandomForestClassifier()
    classifier.fit(counts, train_y)
    print "Predicting..."
    test_count = count_vectorizer.transform(test_text)
    pred = classifier.predict(test_count)
    confusion += confusion_matrix(test_y, pred)
    score = f1_score(test_y, pred, pos_label=SPAM)
    scores.append(score)
print 'Total emails classified: '+ str(len(data))
print 'Score: ' + str(sum(scores)/len(scores))
print 'Confusion matrix:'
print confusion

Fitting...
Predicting...
Fitting...
Predicting...
Fitting...
Predicting...
Fitting...
Predicting...
Fitting...
Predicting...
Fitting...
Predicting...
Total emails classified: 46440
Score: 0.990943483622
Confusion matrix:
[[13352   101]
 [  493 32494]]
